### 기본 라이브러리 불러오기

In [1]:
# 라이브러리 임포트
import os
import pandas as pd
import chardet

# 사용자 정의함수 경로 설정
import sys
sys.path.append("G:/내 드라이브/Source/_Custom_Function")
# 사용자 정의함수 불러오기

import time

import ExcelFile_Merge as em
import ExcelFile_Read as er
import ExcelFile_EDA as ee

In [2]:
df_sum = pd.DataFrame()

In [5]:
# _years = ['2020', '2021', '2022', '2023', '2024']
_years = ['2021']

In [7]:
path_dateset = 'G:\\내 드라이브\\DataSet\\'
path_sub = '서울시 차량통행속도(월별)\\'

---
### 서울시 차량통행속도

- 2020년 1월 서울시 차량통행속도 ~ 2020년 12월 서울시 차량통행속도
- 2021년 1월 서울시 차량통행속도 ~ 2021년 12월 서울시 차량통행속도
- 2022년 1월 서울시 차량통행속도 ~ 2022년 12월 서울시 차량통행속도
- 2023년 1월 서울시 차량통행속도 ~ 2023년 12월 서울시 차량통행속도
- 2024년 1월 서울시 차량통행속도 ~ 2024년 12월 서울시 차량통행속도

In [11]:
for cnt, _year in enumerate(_years):
    
    path = path_dateset + path_sub + _year + '\\'
    savefile = '서울시_차량통행속도_' + _year   # Ex) 서울시_차량통행속도_2024
    print(f'path : {path}, savefile : {savefile}')
    
    # 파일 경로명 변경
    file_list = os.listdir(path)
    file_lists = [file for file in file_list if file.endswith((".csv", ".xlsx"))]
    file_lists.sort()
    print('file_lists : \n', '\n '.join(file_lists))
    print('===========================================')

    # try:
    for _, filename in enumerate(file_lists):        
        print('filename : ', filename + ', encoding : ', em.Get_ExcelEncoding(path + filename))
        start_time_full = time.time()

        # 서울 생활이동 데이터 불러오기
        start_time = time.time()
        print('path + filename : ', path + filename)
        
        dask =  pd.read_excel(path + filename)
        print("Read_dask(" + filename + ") time :", time.time() - start_time)

        df_dask_rtn = dask
        del dask  # 메모리 정리
        
        df_sum = pd.concat([df_dask_rtn, df_sum], axis = 0)
        del df_dask_rtn  # 메모리 정리

    # for문 종료 후, 실행
    df_sum.to_parquet(path + savefile + '.parquet', index=False)
    del df_sum
    
    # except:
    #     print('- Err : ', savefile)  
    

path : G:\내 드라이브\DataSet\서울시 차량통행속도(월별)\2021\, savefile : 서울시_차량통행속도_2021
file_lists : 
 2021년 01월 서울시 차량통행속도.xlsx
 2021년 02월 서울시 차량통행속도.xlsx
 2021년 03월 서울시 차량통행속도.xlsx
 2021년 04월 서울시 차량통행속도.xlsx
 2021년 05월 서울시 차량통행속도.xlsx
 2021년 06월 서울시 차량통행속도.xlsx
 2021년 07월 서울시 차량통행속도.xlsx
 2021년 08월 서울시 차량통행속도.xlsx
 2021년 09월 서울시 차량통행속도.xlsx
 2021년 11월 서울시 차량통행속도.xlsx
 2021년 12월 서울시 차량통행속도.xlsx
filename :  2021년 01월 서울시 차량통행속도.xlsx, encoding :  None
path + filename :  G:\내 드라이브\DataSet\서울시 차량통행속도(월별)\2021\2021년 01월 서울시 차량통행속도.xlsx
Read_dask(2021년 01월 서울시 차량통행속도.xlsx) time : 44.52436971664429
filename :  2021년 02월 서울시 차량통행속도.xlsx, encoding :  None
path + filename :  G:\내 드라이브\DataSet\서울시 차량통행속도(월별)\2021\2021년 02월 서울시 차량통행속도.xlsx
Read_dask(2021년 02월 서울시 차량통행속도.xlsx) time : 41.03913593292236
filename :  2021년 03월 서울시 차량통행속도.xlsx, encoding :  None
path + filename :  G:\내 드라이브\DataSet\서울시 차량통행속도(월별)\2021\2021년 03월 서울시 차량통행속도.xlsx
Read_dask(2021년 03월 서울시 차량통행속도.xlsx) time : 45.498695373535156
filename : 